In [ ]:
!pip install transformers datasets torch nltk rouge-score

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import joblib
import pandas as pd
import os

nltk.download('punkt')

dataset_a_path = "userA_dataset.csv"
dataset_b_path = "userB_dataset.csv"

df_a = pd.read_csv(dataset_a_path)
df_b = pd.read_csv(dataset_b_path)

print(df_a.head(), df_b.head())

conversations = []
for conv_id in df_a['conversation_id'].unique():
    conv_a = df_a[df_a['conversation_id'] == conv_id]
    conv_b = df_b[df_b['conversation_id'] == conv_id]
    merged = pd.concat([conv_a, conv_b]).sort_values('timestamp')
    conversations.append(" ".join(merged['message'].tolist()))

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

class ChatDataset(Dataset):
    def _init_(self, conversations, tokenizer, max_len=512):
        self.examples = []
        for conv in conversations:
            tokenized = tokenizer(conv, truncation=True, max_length=max_len, padding='max_length')
            self.examples.append(tokenized)
    def _len_(self):
        return len(self.examples)
    def _getitem_(self, idx):
        return {key: torch.tensor(val[idx]) if isinstance(val, list) else torch.tensor(val) 
                for key, val in self.examples[idx].items()}

dataset = ChatDataset(conversations, tokenizer)

model = GPT2LMHeadModel.from_pretrained('gpt2')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=1,
    save_steps=200,
    save_total_limit=2,
    logging_steps=50,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

def generate_reply(context, max_length=50):
    input_ids = tokenizer.encode(context + tokenizer.eos_token, return_tensors='pt')
    output_ids = model.generate(input_ids, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    reply = tokenizer.decode(output_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return reply

def evaluate_bleu(reference, candidate):
    reference_tokens = nltk.word_tokenize(reference)
    candidate_tokens = nltk.word_tokenize(candidate)
    return sentence_bleu([reference_tokens], candidate_tokens)

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

context_example = "Hey, are you coming to the meeting today?"
true_reply = "Yes, I will be there in 10 minutes."
pred_reply = generate_reply(context_example)
bleu_score = evaluate_bleu(true_reply, pred_reply)
rouge_score = scorer.score(true_reply, pred_reply)

print("Predicted Reply:", pred_reply)
print("BLEU:", bleu_score)
print("ROUGE:", rouge_score)

joblib.dump(model, "Model.joblib")
print("Model saved as Model.joblib")